In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, BertTokenizerFast, BertForQuestionAnswering
import json
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup
import random
# 檢查是否有可用的GPU
if torch.cuda.is_available():
    print("GPU 可用！")
else:
    print("GPU 不可用！")

GPU 可用！


In [ ]:
import os
os.chdir('/content/drive/MyDrive/bert_qa')

In [ ]:
# %%
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# %%
# download pre-trained model
model_name = 'hfl/chinese-macbert-large'
model = BertForQuestionAnswering.from_pretrained(model_name).to(device)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# read data
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)

    paragraphs_data = []
    questions_data = []
    for item in data['data']:
        for paragraph in item['paragraphs']:
            paragraphs_data.append(paragraph['context'])
            for question in paragraph['qas']:
                questions_data.append({
                    'question': question['question'],
                    'answer': question['answers'][0]['text'],
                    'answer_start': question['answers'][0]['answer_start'],
                    'answer_end': question['answers'][0]['answer_start'] + len(question['answers'][0]['text']) - 1,
                    'paragraph_id':len(paragraphs_data) - 1
                })

    return questions_data, paragraphs_data

train_questions, train_paragraphs = read_data("DRCD_training.json")
dev_questions, dev_paragraphs = read_data("DRCD_dev.json")
test_questions, test_paragraphs = read_data("DRCD_test.json")

# %%
# tokenize
train_questions_tokenized = tokenizer([train_question["question"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question"] for test_question in test_questions], add_special_tokens=False)

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 50
        self.max_paragraph_len = 300

        ##### Change value of doc_stride #####
        self.doc_stride = 200

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # 加入隨機性但一定要包括答案
            random_number = random.randint(10, 150)
            paragraph_start = max(0, min(answer_start_token - random_number, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # A single window is obtained by slicing the portion of paragraph containing the answer
            # mid = (answer_start_token + answer_end_token) // 2
            # paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            # paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]

            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            question_lengths = []

            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):

                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                question_lengths.append(len(tokenized_question.ids[:self.max_question_len]))
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]

                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)

                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)

            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list), question_lengths

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len

        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

In [ ]:
# Function for Evaluation
def evaluate(data, output):

    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    pred_start = 0
    pred_end = 0
    pred_answer = ''
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)

        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob

        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob and start_index <= end_index:
          if end_index.item() - start_index.item() <= 50:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            pred_answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            pred_start = start_index.item()
            pred_end = end_index.item()
          else:
            continue

    # Remove spaces in answer
    return pred_answer.replace(' ',''), pred_start, pred_end

def mapping(paragraph, question_length, start_index, end_index):
    char_start_index = start_index - question_length
    char_end_index = end_index - question_length

    # Extract the answer span from the original paragraph
    predicted_answer = paragraph[char_start_index:char_end_index + 1]

    return predicted_answer

In [ ]:
# training
num_epochs = 1
validation = True
logging_step = 100
initial_lr = 2e-5
optimizer = AdamW(model.parameters(), lr=initial_lr)

total_train_steps = len(train_loader) * num_epochs
num_warmup_steps = int(total_train_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps,
  total_train_steps
)

model.train()

print("Start Training ...")
for epoch in range(num_epochs):
    step = 1
    train_loss = 0
    train_acc = 0

    for data in tqdm(train_loader):
        # Load all data into GPU
        data = [i.to(device) for i in data]

        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)

        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss

        output.loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        step += 1

        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                    attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                ans, start, end = evaluate(data, output)
                dev_acc += (ans == dev_questions[i]["answer"])
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Start Training ...


  3%|▎         | 99/3367 [03:00<1:39:35,  1.83s/it]

Epoch 1 | Step 100 | loss = 5.438, acc = 0.002


  6%|▌         | 199/3367 [06:04<1:37:20,  1.84s/it]

...

 74%|███████▍  | 2499/3367 [1:16:06<26:31,  1.83s/it]

Epoch 1 | Step 2500 | loss = 0.440, acc = 0.805


 77%|███████▋  | 2599/3367 [1:19:09<23:32,  1.84s/it]

Epoch 1 | Step 2600 | loss = 0.466, acc = 0.805


 79%|███████▉  | 2655/3367 [1:20:51<21:43,  1.83s/it]

In [ ]:
# %%
# Save a model and its configuration file to the directory 「saved_model」
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "./model_saved/model_0727_macbert"
model.save_pretrained(model_save_dir)


In [ ]:
# Try model
model_test = BertForQuestionAnswering.from_pretrained('model_saved/model_0726_macbert').to(device)
tokenizer_test = BertTokenizerFast.from_pretrained(model_name)

In [ ]:
# %%
# testing
model.eval()

with torch.no_grad():
    test_acc = 0
    for i, data in enumerate(tqdm(test_loader)):
        output = model_test(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        # prediction is correct only if answer text exactly matches
        ans, start, end = evaluate(data, output)
        test_acc += (ans == test_questions[i]["answer"])

    print(f"acc = {test_acc / len(test_loader):.3f}")

100%|██████████| 3493/3493 [12:22<00:00,  4.70it/s]

acc = 0.824


In [ ]:
example_paragraph = "今天颱風天下雨"
example_question = '今天為什麼下雨'
input = tokenizer_test(example_question, example_paragraph, return_tensors = 'pt')
tokenized_q = tokenizer_test(example_question)
with torch.no_grad():
  output = model_test(**input.to(device))

start = torch.argmax(output.start_logits)
end = torch.argmax(output.end_logits)
char_start = start.item() - len(tokenized_q['input_ids'])
char_end = end.item() - len(tokenized_q['input_ids'])

print('start position:', char_start)
print('end position:', char_end)
predict_id = input['input_ids'][0][start : end + 1]
print('predict id:', predict_id)
predict_answer = tokenizer_test.decode(predict_id).replace(' ','')
answer = example_paragraph[char_start : char_end + 1]
print('predict_answer:', answer)